In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Directive_louis = '/content/drive/MyDrive/Centrale Marseille/3A/SAM/Projet/'
Directive_guillaume ='/content/drive/MyDrive/Centrale Marseille/3A/SAM/'

In [ ]:
!pip install --upgrade --quiet librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 KB 5.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt, matplotlib.image as mpimg
import sys
sys.path.append(Directive_louis)

In [ ]:
import pandas as pd

In [ ]:
import zipfile
with zipfile.ZipFile(Directive_guillaume + 'msdi.zip' , 'r') as zip:
    zip.extractall('msdi')

In [ ]:
mapping = pd.read_csv('msdi/msdi_mapping.csv')
mapping = mapping.sample(frac=1)

In [ ]:
!ls msdi/

deep_features  img  labels.csv	mfcc  msdi_mapping.csv	README.md


In [ ]:
!head msdi/msdi_mapping.csv

,msd_track_id,genre,album_index,set,msd_artist_id,image_url,img,mfcc,deep_features
0,TRABKJU128F422A7FE,Metal,0,train,ARBD4QW1187FB42153,http://artwork-cdn.7static.com/static/img/sleeveart/00/008/487/0000848744_200.jpg,img/0000848744_200.jpg,mfcc/mfcc_A.npz,0
1,TRBLDQQ128F92E58B4,Rock,1,train,AR3RK011187FB3CE3B,http://artwork-cdn.7static.com/static/img/sleeveart/00/004/765/0000476534_200.jpg,img/0000476534_200.jpg,mfcc/mfcc_B.npz,1
2,TRDMMDE128F14A9052,Rock,2,train,ARJVTRE1187B9959C0,http://artwork-cdn.7static.com/static/img/sleeveart/00/000/623/0000062345_200.jpg,img/0000062345_200.jpg,mfcc/mfcc_D.npz,2
3,TRJOPZB128F4250E02,Rock,4,train,AR62BB21187B9AC83D,http://artwork-cdn.7static.com/static/img/sleeveart/00/001/447/0000144785_200.jpg,img/0000144785_200.jpg,mfcc/mfcc_J.npz,3
4,TRJKBVL128F935567B,Rock,5,train,AR7GVOV1187B9B5FF1,http://artwork-cdn.7static.com/static/img/sleeveart/00/005/205/0000520513_200.jpg,img/0000520513_200.jpg,mfcc/mfcc_J.npz,4
5,TRUSOQD128F92CFEA5,Rock,7,train,AR

# Introduction

Ce projet a pour objectif de développer des algorithmes multimodaux pour la prédiction du genre musical d'une chanson en utilisant des informations visuelles, auditives et textuelles. Pour ce faire, nous allons implémenter des algorithmes de fusion tardive et précoce afin de combiner les informations provenant de différentes sources pour obtenir des résultats plus précis. Nous allons également comparer les performances de ces algorithmes multimodaux aux algorithmes unimodaux déjà implémentés dans une partie précédente pour les images et les extraits audio. En utilisant ces algorithmes, nous espérons obtenir une meilleure précision pour la prédiction du genre musical d'une chanson.
L'oganisation de cette partie se découpe comme suit:
##    Traitement des labels:
Cette partie a pour simple objectif de récupérer l'ensemble des différents labels (genres musicaux) et de les encoder en vecteur one-hot afin que les algorithmes de classification puisse les traiter plus simplement.

##    Récupération des différentes features pour chaque type de données:
Pour les données audio, des algorithmes tels que MFCC et deep features sont utilisés pour extraire les caractéristiques du son. Pour les données d'image, les algorithmes tels que l'histogramme de couleur et HOG seront sélectionnés de part leur plus grande performance (qui a été évalué précedemment dans la partie unimodale) pour extraire les caractéristiques de l'image. Enfin, concernant les données textuelles, un algorithmes de BoW (Bag of Words) sera utilisé pour représenter les données textuelles sous forme numérique.

##    Réalisation de la fusion précoce et tardive:
La fusion précoce implique de combiner les caractéristiques des différentes modalités avant l'entraînement de l'algorithme de classification.

Concernant la fusion tardive, une fois que les différentes caractéristiques ont été extraites pour chaque type de données, il est possible de les utiliser pour entraîner des algorithmes de classification préalablement sélectionnés pour chaque modalité. Ensuite, la fusion tardive peut être réalisée en combinant les résultats des algorithmes de chaque modalité pour obtenir une prédiction finale. La fusion précoce implique de combiner les caractéristiques des différentes modalités avant l'entraînement de l'algorithme de classification.

## Traitement des labels

In [ ]:
labels = list(mapping.genre.unique())
print(labels)

['Rock', 'Rap', 'Jazz', 'Reggae', 'World', 'Metal', 'Pop', 'Country', 'Electronic', 'Latin', 'RnB', 'Blues', 'Folk', 'Punk', 'New Age']


In [ ]:
def onehotcode(k, n):
    code = n*[0]
    code[k] = 1
    return code

In [ ]:
def load_labels(rows):
    y = []
    n = len(rows)
    rows.reset_index(drop=True, inplace=True)
    for index, row in rows.iterrows():
        print(f'\rRow {index+1}/{n}', end='')
        y.append(onehotcode(labels.index(row['genre']), len(labels)))
    print()
    return np.vstack(y)

In [ ]:
y_train = mapping[mapping.set == 'train'].genre
y_val = mapping[mapping.set == 'val'].genre
y_test = mapping[mapping.set == 'test'].genre

In [ ]:
print('y_train:',len(y_train),'y_val:',len(y_val),'y_test:',len(y_test))

y_train: 21383 y_val: 4680 y_test: 4649


On utilise le module sci-kit learn pour transformer nos listes de classes en one-hot-code

In [ ]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
y_train_bin = encoder.fit_transform(y_train)
y_val_bin = encoder.transform(y_val)

#Traitement des données

## Traitement des audios

In [ ]:
import numpy as np

def load_audio_deep_features(rows, filename):
    X = []
    n = len(rows)
    rows.reset_index(drop=True, inplace=True)
    npyfile = np.load('msdi/deep_features/' + filename)
    for index, row in rows.iterrows():
        x = npyfile[row['deep_features']]
        X.append(x)
    return np.vstack(X)

In [ ]:
def load_audio_MFCC_features(rows):
    X = []
    n = len(rows)
    rows.reset_index(drop=True, inplace=True)
    for index, row in rows.iterrows():
        npzfile = np.load('msdi/' + row['mfcc'])
        x = npzfile[row['msd_track_id']]
        X.append(aggregate(x))
    return np.vstack(X)

In [ ]:
from librosa.feature import delta

def aggregate(x, delta=0):
    feats = [x]
    for i in range(delta):
        feats.append(delta(x, order=i+1, axis=0))
    return np.hstack(feats).mean(axis=0)

In [ ]:
X_audio_deep_train = load_audio_deep_features(mapping[mapping.set == 'train'], 'X_train_audio_MSD-I.npy')
X_audio_deep_val = load_audio_deep_features(mapping[mapping.set == 'val'], 'X_val_audio_MSD-I.npy')
X_audio_deep_test = load_audio_deep_features(mapping[mapping.set == 'test'], 'X_test_audio_MSD-I.npy')

In [ ]:
X_audio_MFCC_train = load_audio_MFCC_features(mapping[mapping.set == 'train'])
X_audio_MFCC_val = load_audio_MFCC_features(mapping[mapping.set == 'val'])
X_audio_MFCC_test = load_audio_MFCC_features(mapping[mapping.set == 'test'])

## Traitement des images

In [ ]:
from skimage.feature import hog
from skimage import exposure

In [ ]:
P = Q = 3 # taille des blocs (images par blocs)
B = 8 # nombre de bins (directions de gradients considérées)

In [ ]:
# Fonction nous permettant de récupérer les histogrammes par bloc pour chaque image :
def extract_hist(image, P=P, Q=Q, B=B):
    M = (image.shape[0]//P)+1
    N = (image.shape[1]//Q)+1
    tiles = [image[x:x+M,y:y+N] for x in range(0,image.shape[0],M) for y in range(0,image.shape[1],N)]
    return np.concatenate([np.histogram(tile[:,:,c].reshape(-1), bins=B, range=(0, 255))[0] for c in range(image.shape[2]) for tile in tiles])

In [ ]:
# Fonction nous permettant de récupérer le HOG_vector pour chaque image :
def extract_hog(image, P=P, Q=Q, B=B):
  return hog(image, orientations=B, pixels_per_cell=(image.shape[0]//P, image.shape[1]//Q), cells_per_block=(1, 1), multichannel=True)

In [ ]:
def load_image_features(rows):
    hist = []
    hogs = []
    n = len(rows)
    rows.reset_index(drop=True, inplace=True)
    for index, row in rows.iterrows():
        print(f'\rRow {index+1}/{n}', end='')
        img = plt.imread('msdi/' + row['img'])
        hist.append(extract_hist(img))
        hogs.append(extract_hog(img))
               
    print()
    return np.vstack(hist), np.vstack(hogs)

In [ ]:
X_image_hist_train, X_image_hog_train = load_image_features(mapping[mapping.set == 'train'])
X_image_hist_val, X_image_hog_val = load_image_features(mapping[mapping.set == 'val'])
X_image_hist_test, X_image_hog_test = load_image_features(mapping[mapping.set == 'test'])

Row 21383/21383
Row 4680/4680
Row 4649/4649


In [ ]:
from skimage.transform import resize

# fonction qui renvoie l'image des couleurs moyennes par bloc pour chaque image
def extract_mean(image, P=P, Q=Q):
    return resize(image, output_shape=(P, Q), anti_aliasing=True).flatten()

In [ ]:
def load_image_features_mean(rows):
    means = []
    hogs = []
    n = len(rows)
    rows.reset_index(drop=True, inplace=True)
    for index, row in rows.iterrows():
        print(f'\rRow {index+1}/{n}', end='')
        img = plt.imread('msdi/' + row['img'])
        means.append(extract_mean(img))
        hogs.append(extract_hog(img))
               
    print()
    return np.vstack(means), np.vstack(hogs)

In [ ]:
X_image_mean_train, X_image_hog_train = load_image_features_mean(mapping[mapping.set == 'train'])
X_image_mean_val, X_image_hog_val = load_image_features_mean(mapping[mapping.set == 'val'])
X_image_mean_test, X_image_hog_test = load_image_features_mean(mapping[mapping.set == 'test'])

Row 21383/21383
Row 4680/4680
Row 4649/4649


## Traitement des textes

In [ ]:
!wget https://pageperso.lis-lab.fr/benoit.favre/files/msx_lyrics_genre.txt.gz

--2023-02-06 20:12:24--  https://pageperso.lis-lab.fr/benoit.favre/files/msx_lyrics_genre.txt.gz
Resolving pageperso.lis-lab.fr (pageperso.lis-lab.fr)... 139.124.22.27
Connecting to pageperso.lis-lab.fr (pageperso.lis-lab.fr)|139.124.22.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2715032 (2.6M) [application/x-gzip]
Saving to: ‘msx_lyrics_genre.txt.gz’

msx_lyrics_genre.tx 100%[===================>]   2.59M  3.56MB/s    in 0.7s    

2023-02-06 20:12:26 (3.56 MB/s) - ‘msx_lyrics_genre.txt.gz’ saved [2715032/2715032]



In [ ]:
!gzip -d -f msx_lyrics_genre.txt.gz

In [ ]:
!head msx_lyrics_genre.txt

TRAAAHZ128E0799171 Rap 1:39 2:30 3:10 4:10 5:28 6:21 7:1 8:20 9:11 10:12 11:9 12:10 13:9 14:1 15:5 16:11 17:4 18:1 19:1 20:17 22:7 23:4 24:1 26:2 27:2 28:5 29:1 30:6 31:5 32:4 33:2 35:1 36:1 37:8 39:2 40:3 41:2 44:1 45:11 48:16 49:1 50:2 51:2 52:6 54:4 55:3 57:2 58:1 59:1 61:1 62:4 65:1 66:2 69:3 70:1 72:2 76:7 78:3 81:1 82:1 85:2 86:1 88:2 93:2 94:1 95:3 98:1 99:1 100:1 106:3 107:1 108:1 109:1 110:1 111:11 116:1 119:1 133:1 134:2 135:3 136:4 137:1 153:1 155:1 166:1 185:1 190:1 192:1 200:1 201:1 203:2 205:3 206:1 207:1 210:1 216:2 219:1 223:3 228:1 239:1 242:1 244:1 261:1 262:1 277:1 278:1 285:1 289:3 291:3 302:1 304:1 323:1 325:1 327:2 332:6 338:1 339:1 344:1 349:1 355:1 357:3 358:1 363:1 368:1 371:1 378:3 389:1 396:6 410:1 413:1 427:1 431:2 439:1 446:1 448:1 452:1 455:3 456:1 470:1 494:1 500:1 503:1 520:1 525:1 537:1 539:1 546:1 548:1 555:4 578:1 597:1 608:1 615:2 618:3 631:1 646:1 658:2 659:1 663:1 693:1 707:3 712:1 739:1 752:3 811:1 836:2 867:1 868:3 886:2 901:1 918:1 924:1 948:1 1

In [ ]:
bag_of_words = {}
file = open('msx_lyrics_genre.txt')
for line in file.readlines():
    id, _, *words = line.split(' ')
    bag_of_words[id] = dict([map(int, word.split(':')) for word in words])

In [ ]:
max_length = int(max([word_id for words in bag_of_words.values() for word_id in words.keys()]))
max_length

5000

In [ ]:
def load_text_features(rows):
    X = []
    n = len(rows)
    rows.reset_index(drop=True, inplace=True)
    for index, row in rows.iterrows():
        print(f'\rRow {index+1}/{n}', end='')
        x = np.zeros(max_length)
        if row['msd_track_id'] in bag_of_words:
            bag_of_word = bag_of_words[row['msd_track_id']]
            indices = np.array(list(bag_of_word.keys())) - 1
            values = list(bag_of_word.values())
            x[indices] = values
        X.append(x)
    print()
    return np.vstack(X)

In [ ]:
X_text_train = load_text_features(mapping[mapping.set == 'train'])
X_text_val = load_text_features(mapping[mapping.set == 'val'])
X_text_test = load_text_features(mapping[mapping.set == 'test'])

Row 21383/21383
Row 4680/4680
Row 4649/4649


In [ ]:
X_train = [X_audio_deep_train, X_audio_MFCC_train, X_image_hist_train, X_image_hog_train, X_text_train]
X_test = [X_audio_deep_test, X_audio_MFCC_test, X_image_hist_test, X_image_hog_test, X_text_test]
X_val = [X_audio_deep_val, X_audio_MFCC_val, X_image_hist_val, X_image_hog_val, X_text_val]

In [ ]:
X_train = np.hstack(X_train)
X_val = np.hstack(X_val)
X_test = np.hstack(X_test)

In [ ]:
print(X_train.shape)

(21383, 7348)
(21383, 7348)


# Early fusion

In [ ]:
X_train_list = [X_audio_deep_train, X_audio_MFCC_train, X_image_hist_train, X_image_hog_train, X_text_train]
X_test_list = [X_audio_deep_test, X_audio_MFCC_test, X_image_hist_test, X_image_hog_test, X_text_test]
X_val_list = [X_audio_deep_val, X_audio_MFCC_val, X_image_hist_val, X_image_hog_val, X_text_val]

In [ ]:
X_train = np.hstack(X_train_list)
X_val = np.hstack(X_val_list)
X_test = np.hstack(X_test_list)

In [ ]:
from keras import Sequential
from keras.layers import Dense
from keras.losses import categorical_crossentropy

In [ ]:
# Etablissement du modèle
model = Sequential()
model.add(Dense(512, activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(15, activation="softmax"))
model.compile(loss=categorical_crossentropy, metrics=['accuracy'])

In [ ]:
# Entrainement du modèle
model.fit(X_train, y_train_bin, epochs=5, validation_data=(X_val, y_val_bin))

Epoch 1/5
669/669 [==============================] - 36s 51ms/step - loss: 12.4405 - accuracy: 0.1585 - val_loss: 2.5431 - val_accuracy: 0.1543
Epoch 2/5
669/669 [==============================] - 32s 47ms/step - loss: 2.5448 - accuracy: 0.1761 - val_loss: 2.6453 - val_accuracy: 0.1526
Epoch 3/5
669/669 [==============================] - 32s 47ms/step - loss: 2.4878 - accuracy: 0.1738 - val_loss: 2.5327 - val_accuracy: 0.1524
Epoch 4/5
669/669 [==============================] - 33s 49ms/step - loss: 2.4746 - accuracy: 0.1740 - val_loss: 2.5520 - val_accuracy: 0.1509
Epoch 5/5
669/669 [==============================] - 34s 50ms/step - loss: 2.4514 - accuracy: 0.1712 - val_loss: 2.5356 - val_accuracy: 0.1517


Le modèle ne semble pas efficace avec ces features. Il semble que modèle classfiie tous les sons dans la catégorie prédominante du dataset. Un travail a été effectué pour enlever les déséquilibres de distribution des classes du dataset mais aucun résultat n'a été perçu. Essayons avec les features des images aggrégées par moyennes, qui sont des features moins complexes à interpréter. 

In [ ]:
X_train_list = [X_audio_deep_train, X_audio_MFCC_train, X_image_mean_train, X_image_hog_train, X_text_train]
X_test_list = [X_audio_deep_test, X_audio_MFCC_test, X_image_hist_test, X_image_hog_test, X_text_test]
X_val_list = [X_audio_deep_val, X_audio_MFCC_val, X_image_hist_val, X_image_hog_val, X_text_val]

In [ ]:
X_train = np.hstack(X_train_list)
X_val = np.hstack(X_val_list)
X_test = np.hstack(X_test_list)

In [ ]:
# entrainement
model.fit(X_train, y_train_bin, epochs=5, validation_data=(X_val, y_val_bin))

Epoch 1/5
669/669 [==============================] - 27s 39ms/step - loss: 1.4358 - accuracy: 0.5401 - val_loss: 1.5074 - val_accuracy: 0.5203
Epoch 2/5
669/669 [==============================] - 26s 40ms/step - loss: 1.1862 - accuracy: 0.6187 - val_loss: 1.5358 - val_accuracy: 0.5267
Epoch 3/5
669/669 [==============================] - 29s 43ms/step - loss: 1.0703 - accuracy: 0.6613 - val_loss: 1.6823 - val_accuracy: 0.5417
Epoch 4/5
669/669 [==============================] - 26s 39ms/step - loss: 0.9633 - accuracy: 0.6964 - val_loss: 1.8826 - val_accuracy: 0.5226
Epoch 5/5
669/669 [==============================] - 26s 38ms/step - loss: 0.8633 - accuracy: 0.7267 - val_loss: 2.0495 - val_accuracy: 0.5244


In [ ]:
 # prédictions
y_proba = model.predict(X_test) #one-hot-code encoded predictions
y_pred = encoder.inverse_transform(y_proba) #convert back to class names

146/146 [==============================] - 1s 9ms/step


In [ ]:
# évaluation 
from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test, target_names=encoder.classes_))

              precision    recall  f1-score   support

       Blues       0.27      0.43      0.33       121
     Country       0.59      0.36      0.45       316
  Electronic       0.73      0.69      0.71       751
        Folk       0.24      0.17      0.19       193
        Jazz       0.53      0.57      0.55       433
       Latin       0.47      0.44      0.46        86
       Metal       0.63      0.84      0.72       282
     New Age       0.18      0.23      0.20        31
         Pop       0.55      0.44      0.49       573
        Punk       0.23      0.26      0.25        84
         Rap       0.74      0.78      0.76       360
      Reggae       0.49      0.60      0.54       217
         RnB       0.44      0.48      0.46       359
        Rock       0.54      0.54      0.54       826
       World       0.07      0.18      0.10        17

    accuracy                           0.55      4649
   macro avg       0.45      0.47      0.45      4649
weighted avg       0.56   

Le modèle semble fonctionner avec des résultats de bonnes qualités.

#Fusion tardive

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score

In [ ]:
from sklearn.model_selection import cross_val_score

Dans un premier temps nous réalisons les algorithmes unimodaux des différentes features que nous combinerons dans un second temps afin de réaliser la fusion tardive.

## Audios

### MFCC

In [ ]:
pipeline_mfcc = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=9, n_estimators=50))
pipeline_mfcc.fit(np.concatenate((X_audio_MFCC_train, X_audio_MFCC_val), axis=0), np.concatenate((y_train, y_val), axis=0))
val_mfcc = np.mean(cross_val_score(pipeline_mfcc, np.concatenate((X_audio_MFCC_train, X_audio_MFCC_val), axis=0), np.concatenate((y_train, y_val), axis=0), cv=5))


In [ ]:
# accuracy en validation croisée

print(val_mfcc)

0.4194833036659782


In [ ]:
# accuracy en test
acc_mfcc = pipeline_mfcc.score(X_audio_MFCC_test, y_test)
print(acc_mfcc)

0.4177242417724242


In [ ]:
# aire sous la courbe ROC en test
from sklearn.metrics import roc_auc_score
y_proba_mfcc = pipeline_mfcc.predict_proba(X_audio_MFCC_test)
roc_mfcc = roc_auc_score(y_test, y_proba_mfcc, multi_class='ovr')
print(roc_mfcc)

0.8468963827749605


### Deep features

In [ ]:
pipeline_df = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=9, n_estimators=50))
pipeline_df.fit(np.concatenate((X_audio_deep_train, X_audio_deep_val), axis=0), np.concatenate((y_train, y_val), axis=0))
val_df = np.mean(cross_val_score(pipeline_df, np.concatenate((X_audio_deep_train, X_audio_deep_val), axis=0), np.concatenate((y_train, y_val), axis=0), cv=5))


In [ ]:
# accuracy en validation croisée

print(val_df)

0.5054279482237791


In [ ]:
# accuracy en test
acc_df = pipeline_df.score(X_audio_deep_test, y_test)
print(acc_df)

0.49343944934394496


In [ ]:
# aire sous la courbe ROC en test
from sklearn.metrics import roc_auc_score
y_proba_df = pipeline_df.predict_proba(X_audio_deep_test)
roc_df = roc_auc_score(y_test, y_proba_df, multi_class='ovr')
print(roc_df)

0.860756673020489


##Images

### HIST

In [ ]:
pipeline_hist = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=9, n_estimators=50))
pipeline_hist.fit(np.concatenate((X_image_hist_train, X_image_hist_val), axis=0), np.concatenate((y_train, y_val), axis=0))
val_hist = np.mean(cross_val_score(pipeline_hist, np.concatenate((X_image_hist_train, X_image_hist_val), axis=0), np.concatenate((y_train, y_val), axis=0), cv=5))


In [ ]:
# accuracy en validation croisée

print(val_hist)

0.3817628430252664


In [ ]:
# accuracy en test
acc_hist = pipeline_hist.score(X_image_hist_test, y_test)
print(acc_hist)

0.20456012045601205


In [ ]:
# aire sous la courbe ROC en test
from sklearn.metrics import roc_auc_score
y_proba_hist = pipeline_hist.predict_proba(X_image_hist_test)
roc_hist = roc_auc_score(y_test, y_proba_hist, multi_class='ovr')
print(roc_hist)

0.6116941269567001


### HOG

In [ ]:
pipeline_hog = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=9, n_estimators=50))
pipeline_hog.fit(np.concatenate((X_image_hog_train, X_image_hog_val), axis=0), np.concatenate((y_train, y_val), axis=0))


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=9, n_estimators=50))])

In [ ]:
val_hog = np.mean(cross_val_score(pipeline_hog, np.concatenate((X_image_hog_train, X_image_hog_val), axis=0), np.concatenate((y_train, y_val), axis=0), cv=5))

In [ ]:
print(val_hog)

0.2978524819658746


In [ ]:
# accuracy en test
acc_hog = pipeline_hog.score(X_image_hog_test, y_test)
print(acc_hog)

0.19359001935900194


In [ ]:
#ROC en test
from sklearn.metrics import roc_auc_score
y_proba_hog = pipeline_hog.predict_proba(X_image_hog_test)
roc_hog = roc_auc_score(y_test, y_proba_hog, multi_class='ovr')
print(roc_hog)

0.6012921734366118


## Textes

### Bag of Words

In [ ]:
pipeline_bow = make_pipeline(StandardScaler(), RandomForestClassifier(max_depth=9, n_estimators=50))
pipeline_bow.fit(np.concatenate((X_text_train, X_text_val), axis=0), np.concatenate((y_train, y_val), axis=0))


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=9, n_estimators=50))])

In [ ]:
val_bow = np.mean(cross_val_score(pipeline_bow, np.concatenate((X_text_train, X_text_val), axis=0), np.concatenate((y_train, y_val), axis=0), cv=5))

In [ ]:
print(val_bow)

0.2907180731682218


In [ ]:
# accuracy en test
acc_bow = pipeline_bow.score(X_text_test, y_test)
print(acc_bow)

0.2903850290385029


In [ ]:
#ROC en test
y_proba_bow = pipeline_bow.predict_proba(X_text_test)
roc_bow = roc_auc_score(y_test, y_proba_bow, multi_class='ovr')
print(roc_bow)

0.695429584432678


## Fusion tardive à votes non pondérés

In [ ]:
# liste des prédictions des estimateurs 1 à 1
predictions = np.transpose(np.array([pipeline_mfcc.predict(X_audio_MFCC_test), pipeline_df.predict(X_audio_deep_test), pipeline_hist.predict(X_image_hist_test), pipeline_hog.predict(X_image_hog_test), pipeline_bow.predict(X_text_test)]))
print(np.shape(predictions))

(4649, 5)


In [ ]:
# prediction du vote
dx, dy = np.shape(predictions)[0], np.shape(predictions)[1]
occurences = []
y_pred_vote_nonpond = []

for i in range(dx):
  occurences = [predictions[i, :].tolist().count(predictions[i, j]) for j in range(dy)]
  k = np.argmax(occurences) # dans le cas où il y a autant de votes majoritaires pour plusieurs classes, c'est la 1ère de ces classes qui est sélectionnée
  y_pred_vote_nonpond.append(predictions[i, k])
 
# accuracy en test
from sklearn.metrics import accuracy_score
acc_vote_nonpond = accuracy_score(y_test, y_pred_vote_nonpond)
print('accuracy du vote à la majorité sans pondération en test :', acc_vote_nonpond)

accuracy du vote à la majorité sans pondération en test : 0.4099806409980641


In [ ]:
# Probabilités de chaque classe
y_probas = [y_proba_mfcc, y_proba_df, y_proba_hist, y_proba_hog, y_proba_bow]
print(np.shape(y_probas))

(5, 4649, 15)


In [ ]:
# prediction probas classes
y_proba_uni = np.mean(y_probas, axis=0)
print(np.shape(y_proba_uni))
 

(4649, 15)


In [ ]:
from sklearn.metrics import roc_auc_score
roc_nonpond = roc_auc_score(y_test, y_proba_uni, multi_class='ovr')
print(roc_nonpond)

0.9031383074912048


## Fusion tardive à votes pondérés

In [ ]:
# prediction classe
y_pred_uni = pipeline_bow.classes_[np.argmax(y_proba_uni, axis=1)]
print(np.shape(y_pred_uni))
 
# accuracy 
from sklearn.metrics import accuracy_score
acc_uni = accuracy_score(y_test, y_pred_uni)
print(acc_uni)
 
# roc score 
from sklearn.metrics import roc_auc_score
roc_uni = roc_auc_score(y_test, y_proba_uni, multi_class='ovr')
print(roc_uni)

(4649, 15)
(4649,)
0.46612174661217465
0.9031383074912048


In [ ]:
# pondération donnée par l'accuracy obtenue en validation croisée
pond = [val_mfcc, val_df, val_hist, val_hog, val_bow]
 
# prediction probas classes
y_proba_pond = np.average(y_probas, axis=0, weights=pond)
 
# prediction classe
y_pred_pond = pipeline_bow.classes_[np.argmax(y_proba_pond, axis=1)]
 
# accuracy en test
from sklearn.metrics import accuracy_score
acc_vote_pond = accuracy_score(y_test, y_pred_pond)
print('accuracy du vote à la majorité avec pondération en test :', acc_vote_pond)
 


accuracy du vote à la majorité avec pondération en test : 0.48375994837599484


In [ ]:
# roc score en test
from sklearn.metrics import roc_auc_score
roc_pond = roc_auc_score(y_test, y_proba_pond, multi_class='ovr')
print('score ROC en test :', roc_pond)

score ROC en test : 0.9026313415384922


En utilisant une pondération, les algorithmes ayant une meilleure performance en validation croisée vont avoir plus de poids lors de la prise de décision finale pour faire des prédictions sur les données de test. Cela peut aider à éviter l'overfitting sur les données d'entraînement en permettant à plusieurs algorithmes de contribuer à la prédiction finale. De plus, cela peut aider à améliorer la précision de la prédiction en utilisant les forces de différents algorithmes pour compléter les faiblesses de chacun. Cela expliquerait pourquoi la fusion pondérée obtient une meilleure précision que la fusion sans pondération.


# Conclusion

| Type | Features | Accuracy | ROC |
|------|----------|----------|----------|
| Unimodal | Son (MFCC)| 41,7% | 84,6 |
| Unimodal | Son (Deep features)| 49% | 86% |
| Unimodal | Image (histogramme)| 20,5% | 61,1% |
| Unimodal | Image (HOG) | 19,4% | 60,1% |
| Unimodal | Texte (BoW) | 29% | 69,5% |
| Multimodal | Early-fusion | 0.45% | - |
| Multimodal | Late-fusion à votes non pondérés | 41% | 90,3% |
| Multimodal | Late-fusion à votes pondérés| 48,4% | 90,3% |



En conclusion, ce projet a exploré différents algorithmes multimodaux et unimodaux pour prédire le genre d'une musique à partir d'une image de couverture d'album, d'un extrait audio et de données textuelles. Les résultats montrent que les algorithmes de traitement du son sont les plus performants en termes de précision de classification avec un score de 49% pour les features de deep learning du son. Les algorithmes de traitement d'image ont eu des performances inférieures avec des scores de 20,5% pour l'histogramme de couleur et 19,4% pour HOG. Le traitement du texte a également donné des résultats inférieurs avec une précision de 29% pour l'utilisation de bag-of-words.

Il y a plusieurs raisons qui pourraient expliquer pourquoi les données de son ont permis une plus grande précision que les données d'image et de texte pour la classification du genre musical.

Tout d'abord, le son est un aspect clé de la musique et peut être utilisé pour capturer les caractéristiques distinctives du genre, telles que la fréquence, le tempo, les harmoniques et la timbre. Les algorithmes de traitement du son tels que les features MFCC et les features de deep learning peuvent extraire ces caractéristiques et les utiliser pour la classification.

En comparaison, l'image de la couverture de l'album peut être influencée par de nombreux facteurs tels que le design, la couleur et les textes, ce qui peut rendre plus difficile la prédiction du genre musical car le lien peut souvent être très indirect. De même, les données textuelles telles que les titres de chansons et les informations de l'artiste peuvent être influencées par de nombreux facteurs tels que les erreurs de saisie et la subjectivité des étiquetages, ce qui peut également rendre la prédiction du genre musical difficile.

Enfin, il est important de noter que les algorithmes de traitement du son ont été améliorés grâce à des années de recherche et de développement, ce qui a permis d'optimiser leur performance pour la classification du genre musical.

En résumé, la performance supérieure des données de son pour la classification du genre musical peut être attribuée à la capacité de ces algorithmes à capturer les caractéristiques sonores distinctives de la musique, ainsi qu'à l'amélioration continue de ces algorithmes.

Les algorithmes de fusion tardive (late-fusion) ont également été testés avec des votes non pondérés et pondérés. Les résultats montrent que la fusion tardive pondérée a donné les meilleurs résultats avec une précision de 48,4% et une courbe ROC de 90,3%. Cependant, il est important de noter que les résultats de la fusion tardive ne sont que la somme des résultats des algorithmes unimodaux, il est donc nécessaire d'optimiser les algorithmes unimodaux pour améliorer les performances globales.

En général, ce projet montre que la prédiction du genre musical peut être un défi complexe et nécessite une combinaison judicieuse de différentes features pour obtenir des résultats satisfaisants. Il est donc nécessaire de continuer à explorer et à améliorer les algorithmes multimodaux pour optimiser la prédiction du genre musical.